### Shedding
In emergency conditions, it may be possible / necessary for a grid operator to disconnect certain loads, generators, or other components in order to prevent a larger blackout. This notebook explores how this can be achieved in Grid2OP. 

By default shedding is disabled in all environments, to enable it provide set the environment parameter ALLOW_SHEDDING to True.

Shed load:
* **Vector** :: $\text{n\_load}$
* **List** :: $[(\text{load\_id}, \text{status})]$
* **Dictionary** :: $\text{load\_name}: \text{status}$

Shed generator:
* **Vector** :: $\text{n\_gen}$
* **List** :: $[(\text{gen\_id}, \text{status})]$
* **Dictionary** :: $\text{gen\_name}: \text{status}$

Where $\text{status}$ is a boolean (True/False).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import grid2op
from grid2op.Parameters import Parameters

from grid2op.PlotGrid import PlotMatplot
from pathlib import Path


p = Parameters()
p.MAX_SUB_CHANGED = 5

data_path = Path.cwd() / "grid2op" / "data"
env = grid2op.make(data_path / "rte_case5_example", allow_shedding=True, n_busbar=3, param=p)
plotter = PlotMatplot(env.observation_space, load_name=True, gen_name=True, dpi=150)
env.set_id("00")
obs = env.reset()

print(f"Loads: {env.n_load}, Generators: {env.n_gen}, Storage: {env.n_storage}")
plotter.plot_obs(obs, figure=plt.figure(figsize=(8,5)))
plt.show()

In [ ]:
load_lookup = {name:i for i,name in enumerate(env.name_load)}
act = env.action_space({"set_bus":[(env.load_pos_topo_vect[load_lookup["load_4_2"]], -1)]})
print(act)
env.set_id("00")
init_obs = env.reset()
obs, reward, done, info = env.step(act)
plotter.plot_obs(obs, figure=plt.figure(figsize=(8,5)))
plt.show()

In [ ]:
load_shed_mask = (init_obs + act).topo_vect[env.load_pos_topo_vect]
gen_shed_mask = (init_obs + act).topo_vect[env.gen_pos_topo_vect]

In [ ]:
import pandapower as pp
from pandapower.plotting import simple_plot
network = env.backend._grid.deepcopy()
display(network.res_line.loc[:, ["p_from_mw", "p_to_mw", "q_from_mvar", "q_to_mvar"]])
pp.runpp(network,
         check_connectivity=False,
         init="dc",
         lightsim2grid=False,
         max_iteration=10,
         distributed_slack=False,
)
display(network.res_line.loc[:, ["p_from_mw", "p_to_mw", "q_from_mvar", "q_to_mvar"]])